In [1]:
# @title Cell 1: Bootstrap CI Configuration - Load ViT AF Evaluation Results

# File: 10_2_Bootstrap_CI_ViT_AF_Cell1.py
# Location: experiments/10_2_Bootstrap_CI_ViT_AF.ipynb
# Purpose: Load ViT-patch32 AF predictions from existing evaluation results for bootstrap CI

import os
import json
import pickle
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive

print("=" * 70)
print("BOOTSTRAP CONFIDENCE INTERVALS FOR VIT-PATCH32 AF")
print("Statistical Validation via Resampling Methods")
print("=" * 70)

# =====================================================
# SECTION 1: ENVIRONMENT CONFIGURATION
# =====================================================

print("\n[STEP 1] Mounting Google Drive and configuring environment...")
drive.mount('/content/drive')

# Project structure configuration
PROJECT_ROOT = "/content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project"
EXPERIMENT_ID = "run_02_patch32_FL"

# Path to existing evaluation results (AF = apex-only test set)
EVALUATION_RESULTS_PATH = f"{PROJECT_ROOT}/results/02_01_vit_casme2-af/{EXPERIMENT_ID}/evaluation_results/casme2_vit_direct_evaluation_results.json"
RESULTS_ROOT = f"{PROJECT_ROOT}/results/bootstrap_ci_results"

# Create results directory
os.makedirs(RESULTS_ROOT, exist_ok=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"Loading from: casme2_vit_direct_evaluation_results.json (Apex-Only Test Set)")
print(f"Results output: bootstrap_ci_results/")

# Verify evaluation results file exists
if not os.path.exists(EVALUATION_RESULTS_PATH):
    raise FileNotFoundError(f"Evaluation results not found: {EVALUATION_RESULTS_PATH}")
print("Evaluation results file verified: exists")

# CASME II configuration
CASME2_CLASSES = ['others', 'disgust', 'happiness', 'repression', 'surprise', 'sadness', 'fear']
CLASS_TO_IDX = {cls: idx for idx, cls in enumerate(CASME2_CLASSES)}
NUM_CLASSES = 7

# =====================================================
# SECTION 2: LOAD EVALUATION RESULTS
# =====================================================

print("\n[STEP 2] Loading ViT evaluation results from existing JSON...")

with open(EVALUATION_RESULTS_PATH, 'r') as f:
    eval_results = json.load(f)

# Extract metadata
eval_metadata = eval_results['evaluation_metadata']
overall_perf = eval_results['overall_performance']
confusion_matrix = np.array(eval_results['confusion_matrix'])

print(f"Evaluation results loaded successfully")
print(f"  Model: ViT-patch32")
print(f"  Phase: AF (Apex Frame)")
print(f"  Test samples: {eval_metadata['test_samples']}")
print(f"  Original Macro F1: {overall_perf['macro_f1']:.4f}")
print(f"  Original Accuracy: {overall_perf['accuracy']:.4f}")

# Identify available classes
available_classes = eval_metadata['available_classes']
missing_classes = eval_metadata['missing_classes']

print(f"  Available classes: {len(available_classes)}")
print(f"  Missing classes: {missing_classes}")

# =====================================================
# SECTION 3: RECONSTRUCT PREDICTIONS FROM CONFUSION MATRIX
# =====================================================

print("\n[STEP 3] Reconstructing predictions from confusion matrix...")

def reconstruct_predictions_from_confusion_matrix(cm, class_names):
    """
    Reconstruct y_true and y_pred arrays from confusion matrix

    Args:
        cm: Confusion matrix (true labels × predicted labels)
        class_names: List of class names

    Returns:
        y_true, y_pred: Arrays of true and predicted labels
    """
    y_true = []
    y_pred = []

    # For each true class (rows)
    for true_idx in range(len(class_names)):
        # For each predicted class (columns)
        for pred_idx in range(len(class_names)):
            count = int(cm[true_idx, pred_idx])
            # Add 'count' samples with this true→pred mapping
            y_true.extend([true_idx] * count)
            y_pred.extend([pred_idx] * count)

    return np.array(y_true), np.array(y_pred)

# Reconstruct predictions
y_true, y_pred = reconstruct_predictions_from_confusion_matrix(confusion_matrix, CASME2_CLASSES)

print(f"Predictions reconstructed successfully")
print(f"  Total samples: {len(y_true)}")
print(f"  Unique true labels: {sorted(np.unique(y_true))}")
print(f"  Unique predictions: {sorted(np.unique(y_pred))}")

# =====================================================
# SECTION 4: VERIFY RECONSTRUCTED PREDICTIONS
# =====================================================

print("\n[STEP 4] Verifying reconstructed predictions...")

from sklearn.metrics import f1_score, accuracy_score, precision_recall_fscore_support

# Calculate metrics on reconstructed predictions
test_accuracy = accuracy_score(y_true, y_pred)

# Identify classes present in test set
unique_test_labels = sorted(np.unique(y_true))

# Macro metrics - ONLY for classes present in test set
precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
    y_true, y_pred,
    average='macro',
    zero_division=0,
    labels=unique_test_labels
)

print("Reconstructed metrics:")
print(f"  Accuracy: {test_accuracy:.4f}")
print(f"  Macro Precision: {precision_macro:.4f}")
print(f"  Macro Recall: {recall_macro:.4f}")
print(f"  Macro F1: {f1_macro:.4f}")

# Verification against original results
EXPECTED_F1 = overall_perf['macro_f1']
EXPECTED_ACC = overall_perf['accuracy']

f1_diff = abs(f1_macro - EXPECTED_F1)
acc_diff = abs(test_accuracy - EXPECTED_ACC)

print("\nVerification check:")
print(f"  Expected F1: {EXPECTED_F1:.4f}")
print(f"  Calculated F1: {f1_macro:.4f}")
print(f"  Difference: {f1_diff:.6f}")

if f1_diff < 0.0001:
    print("  Status: VERIFIED - Metrics match reported results perfectly")
elif f1_diff < 0.001:
    print("  Status: VERIFIED - Metrics match reported results (minor rounding)")
else:
    print(f"  Warning: Metrics differ by {f1_diff:.6f}")

print(f"\nExpected Accuracy: {EXPECTED_ACC:.4f}")
print(f"Calculated Accuracy: {test_accuracy:.4f}")
print(f"Difference: {acc_diff:.6f}")

# Per-class verification
per_class_perf = eval_results['per_class_performance']

# Calculate per-class F1 properly
precision_per_class, recall_per_class, f1_per_class, support_per_class = precision_recall_fscore_support(
    y_true, y_pred,
    average=None,
    zero_division=0,
    labels=list(range(NUM_CLASSES))
)

print("\nPer-class F1 scores (verification):")
for i, class_name in enumerate(CASME2_CLASSES):
    in_test = i in unique_test_labels
    status = "present" if in_test else "missing"

    calculated_f1 = f1_per_class[i]
    expected_f1 = per_class_perf[class_name]['f1_score']
    support = int(support_per_class[i])

    print(f"  {class_name} [{status}]: F1={calculated_f1:.4f} (expected: {expected_f1:.4f}), Support={support}")

# =====================================================
# SECTION 5: PREPARE METADATA
# =====================================================

print("\n[STEP 5] Preparing comprehensive metadata...")

# Class distribution
test_dist = {}
for i, class_name in enumerate(CASME2_CLASSES):
    count = int(np.sum(y_true == i))
    test_dist[class_name] = count

print("\nTest set distribution:")
for class_name in CASME2_CLASSES:
    count = test_dist[class_name]
    percentage = (count / len(y_true) * 100) if len(y_true) > 0 else 0
    print(f"  {class_name}: {count} samples ({percentage:.1f}%)")

# =====================================================
# SECTION 6: SAVE PREDICTIONS FOR BOOTSTRAP
# =====================================================

print("\n[STEP 6] Saving predictions for bootstrap analysis...")

# Prepare data structure for bootstrap
bootstrap_data = {
    'metadata': {
        'model': 'ViT-patch32',
        'methodology': 'M1 (Raw Images)',
        'phase': 'AF (Apex Frame)',
        'experiment_id': EXPERIMENT_ID,
        'test_dataset': 'data_split_v1',
        'test_dataset_description': 'Phase 1 Apex-Only (Peak Single-Frame)',
        'test_samples': int(len(y_true)),
        'num_classes': NUM_CLASSES,
        'class_names': CASME2_CLASSES,
        'available_classes': available_classes,
        'missing_classes': missing_classes,
        'data_source': 'reconstructed_from_evaluation_results',
        'evaluation_timestamp': eval_metadata['evaluation_timestamp'],
        'bootstrap_timestamp': datetime.now().strftime("%Y%m%d_%H%M%S")
    },
    'predictions': {
        'y_true': y_true.tolist(),
        'y_pred': y_pred.tolist(),
        'reconstruction_method': 'confusion_matrix',
        'original_confusion_matrix': confusion_matrix.tolist()
    },
    'metrics': {
        'accuracy': float(test_accuracy),
        'macro_precision': float(precision_macro),
        'macro_recall': float(recall_macro),
        'macro_f1': float(f1_macro),
        'macro_calculation_note': 'Macro metrics calculated only for classes present in test set',
        'class_distribution': test_dist,
        'original_metrics': overall_perf
    },
    'training_info': eval_results['training_information']
}

# Save as JSON
json_path = f"{RESULTS_ROOT}/vit_patch32_af_predictions.json"
with open(json_path, 'w') as f:
    json.dump(bootstrap_data, f, indent=2)

print(f"Predictions saved to JSON: {os.path.basename(json_path)}")

# Save as pickle for fast loading in Cell 2
pickle_path = f"{RESULTS_ROOT}/vit_patch32_af_predictions.pkl"
with open(pickle_path, 'wb') as f:
    pickle.dump(bootstrap_data, f)

print(f"Predictions saved to pickle: {os.path.basename(pickle_path)}")

# Verification
file_size_json = os.path.getsize(json_path) / 1024
file_size_pkl = os.path.getsize(pickle_path) / 1024
print(f"  JSON file size: {file_size_json:.1f} KB")
print(f"  Pickle file size: {file_size_pkl:.1f} KB")

# =====================================================
# SECTION 7: SUMMARY AND NEXT STEPS
# =====================================================

print("\n" + "=" * 70)
print("CELL 1 COMPLETED: PREDICTIONS LOADED AND VERIFIED")
print("=" * 70)

print("\nSummary:")
print(f"  Model: ViT-patch32 AF (Peak Single-Frame)")
print(f"  Test dataset: Apex-Only (Phase 1, v1)")
print(f"  Test samples: {len(y_true)}")
print(f"  Macro F1: {f1_macro:.4f}")
print(f"  Accuracy: {test_accuracy:.4f}")
print(f"  Verification: PASSED")

print("\nOutput files:")
print(f"  1. {os.path.basename(json_path)}")
print(f"  2. {os.path.basename(pickle_path)}")

print("\nNext step:")
print("  Run Cell 2 to perform bootstrap confidence interval analysis")
print("  Expected bootstrap CI for F1 = 0.4235 with n=28 samples")

print("\n" + "=" * 70)

BOOTSTRAP CONFIDENCE INTERVALS FOR VIT-PATCH32 AF
Statistical Validation via Resampling Methods

[STEP 1] Mounting Google Drive and configuring environment...
Mounted at /content/drive
Project root: /content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project
Loading from: casme2_vit_direct_evaluation_results.json (Apex-Only Test Set)
Results output: bootstrap_ci_results/
Evaluation results file verified: exists

[STEP 2] Loading ViT evaluation results from existing JSON...
Evaluation results loaded successfully
  Model: ViT-patch32
  Phase: AF (Apex Frame)
  Test samples: 28
  Original Macro F1: 0.4235
  Original Accuracy: 0.5357
  Available classes: 6
  Missing classes: ['fear']

[STEP 3] Reconstructing predictions from confusion matrix...
Predictions reconstructed successfully
  Total samples: 28
  Unique true labels: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5)]
  Unique predictions: [np.int64(0), np.int64(1), np.int64(2), np.int64(3

In [2]:
# @title Cell 2: Bootstrap Confidence Interval Analysis

# File: 10_2_Bootstrap_CI_ViT_AF_Cell2.py
# Location: experiments/10_2_Bootstrap_CI_ViT_AF.ipynb
# Purpose: Calculate bootstrap confidence intervals for ViT-patch32 AF macro F1 score

import os
import json
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_recall_fscore_support
import warnings
warnings.filterwarnings('ignore')

print("=" * 70)
print("BOOTSTRAP CONFIDENCE INTERVAL ANALYSIS")
print("Resampling-Based Statistical Validation - ViT-patch32 AF")
print("=" * 70)

# =====================================================
# SECTION 1: CONFIGURATION AND DATA LOADING
# =====================================================

print("\n[STEP 1] Loading predictions from Cell 1...")

# Path configuration
PROJECT_ROOT = "/content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project"
RESULTS_ROOT = f"{PROJECT_ROOT}/results/bootstrap_ci_results"
PREDICTIONS_PATH = f"{RESULTS_ROOT}/vit_patch32_af_predictions.pkl"

# Verify predictions file exists
if not os.path.exists(PREDICTIONS_PATH):
    raise FileNotFoundError(f"Predictions file not found: {PREDICTIONS_PATH}")

# Load predictions from Cell 1
with open(PREDICTIONS_PATH, 'rb') as f:
    bootstrap_data = pickle.load(f)

# Extract data
y_true = np.array(bootstrap_data['predictions']['y_true'])
y_pred = np.array(bootstrap_data['predictions']['y_pred'])
metadata = bootstrap_data['metadata']
original_metrics = bootstrap_data['metrics']

print(f"Predictions loaded successfully")
print(f"  Model: {metadata['model']}")
print(f"  Phase: {metadata['phase']}")
print(f"  Test samples: {len(y_true)}")
print(f"  Original Macro F1: {original_metrics['macro_f1']:.4f}")

# Identify available classes (exclude classes with zero support)
unique_labels = sorted(np.unique(y_true))
available_classes = metadata['available_classes']
missing_classes = metadata['missing_classes']

print(f"  Available classes: {len(available_classes)}")
print(f"  Missing classes: {missing_classes}")

# =====================================================
# SECTION 2: BOOTSTRAP FUNCTION IMPLEMENTATION
# =====================================================

print("\n[STEP 2] Implementing bootstrap resampling function...")

def bootstrap_confidence_interval(y_true, y_pred, n_iterations=1000, confidence=0.95, seed=42):
    """
    Calculate bootstrap confidence intervals for macro F1 score

    Bootstrap resampling methodology:
    1. Resample test set with replacement (same size as original)
    2. Calculate macro F1 on resampled data (only for available classes)
    3. Repeat n_iterations times
    4. Calculate percentile-based confidence intervals

    Args:
        y_true: Ground truth labels
        y_pred: Model predictions
        n_iterations: Number of bootstrap iterations (default: 1000)
        confidence: Confidence level (default: 0.95 for 95% CI)
        seed: Random seed for reproducibility

    Returns:
        dict: Bootstrap results with CI bounds, mean, std, and distribution
    """
    np.random.seed(seed)

    n_samples = len(y_true)
    bootstrap_scores = []

    # Identify available classes in original data
    unique_labels = sorted(np.unique(y_true))

    print(f"Bootstrap configuration:")
    print(f"  Iterations: {n_iterations}")
    print(f"  Confidence level: {confidence * 100:.0f}%")
    print(f"  Sample size: {n_samples}")
    print(f"  Random seed: {seed}")
    print(f"  Metric: Macro F1 (available classes only)")

    # Bootstrap iterations
    for i in tqdm(range(n_iterations), desc="Bootstrap resampling"):
        # Resample with replacement
        indices = np.random.choice(n_samples, size=n_samples, replace=True)
        y_true_boot = y_true[indices]
        y_pred_boot = y_pred[indices]

        # Identify available classes in this bootstrap sample
        # (may differ from original if some classes not sampled)
        unique_boot = sorted(np.unique(y_true_boot))

        # Calculate macro F1 only for available classes
        if len(unique_boot) > 0:
            _, _, f1_boot, _ = precision_recall_fscore_support(
                y_true_boot, y_pred_boot,
                average='macro',
                labels=unique_boot,
                zero_division=0
            )
            bootstrap_scores.append(f1_boot)
        else:
            # Edge case: empty bootstrap sample (extremely rare)
            bootstrap_scores.append(0.0)

    bootstrap_scores = np.array(bootstrap_scores)

    # Calculate confidence interval using percentile method
    alpha = (1 - confidence) / 2
    lower_percentile = alpha * 100
    upper_percentile = (1 - alpha) * 100

    ci_lower = np.percentile(bootstrap_scores, lower_percentile)
    ci_upper = np.percentile(bootstrap_scores, upper_percentile)
    ci_mean = np.mean(bootstrap_scores)
    ci_std = np.std(bootstrap_scores)
    ci_median = np.median(bootstrap_scores)

    results = {
        'confidence_interval': {
            'lower': float(ci_lower),
            'upper': float(ci_upper),
            'confidence_level': confidence
        },
        'statistics': {
            'mean': float(ci_mean),
            'median': float(ci_median),
            'std': float(ci_std),
            'min': float(np.min(bootstrap_scores)),
            'max': float(np.max(bootstrap_scores))
        },
        'bootstrap_distribution': bootstrap_scores.tolist(),
        'parameters': {
            'n_iterations': n_iterations,
            'n_samples': n_samples,
            'seed': seed
        }
    }

    return results

print("Bootstrap function implemented")
print("  Method: Percentile-based confidence intervals")
print("  Resampling: With replacement, preserving sample size")

# =====================================================
# SECTION 3: RUN BOOTSTRAP ANALYSIS
# =====================================================

print("\n[STEP 3] Running bootstrap analysis...")
print("This may take 10-20 seconds for 1000 iterations")

# Run bootstrap with standard parameters
bootstrap_results = bootstrap_confidence_interval(
    y_true=y_true,
    y_pred=y_pred,
    n_iterations=1000,
    confidence=0.95,
    seed=42
)

# Extract results
ci_lower = bootstrap_results['confidence_interval']['lower']
ci_upper = bootstrap_results['confidence_interval']['upper']
ci_mean = bootstrap_results['statistics']['mean']
ci_std = bootstrap_results['statistics']['std']
ci_median = bootstrap_results['statistics']['median']

print("\nBootstrap analysis completed")
print(f"  Bootstrap mean F1: {ci_mean:.4f}")
print(f"  Bootstrap std: {ci_std:.4f}")
print(f"  95% CI: [{ci_lower:.4f}, {ci_upper:.4f}]")
print(f"  CI width: {ci_upper - ci_lower:.4f}")

# Compare with original metric
original_f1 = original_metrics['macro_f1']
bias = ci_mean - original_f1

print(f"\nComparison with original:")
print(f"  Original F1: {original_f1:.4f}")
print(f"  Bootstrap mean: {ci_mean:.4f}")
print(f"  Bias: {bias:+.6f}")

if abs(bias) < 0.01:
    print(f"  Assessment: Low bias, bootstrap distribution is centered")
else:
    print(f"  Assessment: Moderate bias detected")

# =====================================================
# SECTION 4: STATISTICAL INTERPRETATION
# =====================================================

print("\n[STEP 4] Statistical interpretation...")

# Calculate key statistics
ci_width = ci_upper - ci_lower
relative_ci_width = (ci_width / original_f1) * 100
margin_of_error = ci_width / 2

print("Confidence interval analysis:")
print(f"  Point estimate (original): {original_f1:.4f}")
print(f"  95% CI: [{ci_lower:.4f}, {ci_upper:.4f}]")
print(f"  Margin of error: ±{margin_of_error:.4f}")
print(f"  Relative CI width: {relative_ci_width:.1f}% of point estimate")

# Stability assessment
if ci_width < 0.10:
    stability = "High stability"
    interpretation = "Narrow confidence interval indicates robust performance"
elif ci_width < 0.15:
    stability = "Moderate stability"
    interpretation = "Reasonable confidence interval for small test set"
else:
    stability = "Low stability"
    interpretation = "Wide confidence interval reflects test set size limitations"

print(f"\nStability assessment: {stability}")
print(f"  {interpretation}")

# Statistical significance heuristics
lower_bound_threshold = 0.35
if ci_lower > lower_bound_threshold:
    print(f"\nPerformance reliability:")
    print(f"  Lower bound ({ci_lower:.4f}) exceeds {lower_bound_threshold:.2f} threshold")
    print(f"  Conclusion: Consistently above baseline with 95% confidence")
else:
    print(f"\nPerformance reliability:")
    print(f"  Lower bound ({ci_lower:.4f}) near or below {lower_bound_threshold:.2f} threshold")
    print(f"  Conclusion: Performance variability due to small test set")

# =====================================================
# SECTION 5: VISUALIZATION
# =====================================================

print("\n[STEP 5] Creating distribution visualization...")

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Subplot 1: Bootstrap distribution histogram
bootstrap_distribution = np.array(bootstrap_results['bootstrap_distribution'])

ax1.hist(bootstrap_distribution, bins=40, color='steelblue', alpha=0.7, edgecolor='black')
ax1.axvline(original_f1, color='red', linestyle='--', linewidth=2, label=f'Original F1: {original_f1:.4f}')
ax1.axvline(ci_lower, color='green', linestyle='--', linewidth=1.5, label=f'95% CI Lower: {ci_lower:.4f}')
ax1.axvline(ci_upper, color='green', linestyle='--', linewidth=1.5, label=f'95% CI Upper: {ci_upper:.4f}')
ax1.axvline(ci_mean, color='orange', linestyle='-', linewidth=2, label=f'Bootstrap Mean: {ci_mean:.4f}')

ax1.set_xlabel('Macro F1 Score', fontsize=12, fontweight='bold')
ax1.set_ylabel('Frequency', fontsize=12, fontweight='bold')
ax1.set_title('Bootstrap Distribution of Macro F1 Score\nViT-patch32 AF (n=1000)',
              fontsize=13, fontweight='bold', pad=15)
ax1.legend(loc='upper left', fontsize=10, frameon=True, shadow=True)
ax1.grid(True, alpha=0.3)

# Subplot 2: Box plot with confidence interval
box_data = [bootstrap_distribution]
bp = ax2.boxplot(box_data, vert=True, patch_artist=True, widths=0.5,
                 boxprops=dict(facecolor='lightblue', alpha=0.7),
                 medianprops=dict(color='red', linewidth=2),
                 whiskerprops=dict(color='black', linewidth=1.5),
                 capprops=dict(color='black', linewidth=1.5))

ax2.axhline(original_f1, color='red', linestyle='--', linewidth=2, label=f'Original F1: {original_f1:.4f}')
ax2.axhline(ci_lower, color='green', linestyle='--', linewidth=1.5, alpha=0.7, label='95% CI Bounds')
ax2.axhline(ci_upper, color='green', linestyle='--', linewidth=1.5, alpha=0.7)

ax2.set_ylabel('Macro F1 Score', fontsize=12, fontweight='bold')
ax2.set_title('Bootstrap Distribution Summary\nwith 95% Confidence Interval',
              fontsize=13, fontweight='bold', pad=15)
ax2.set_xticks([1])
ax2.set_xticklabels(['Bootstrap Samples'], fontsize=11)
ax2.legend(loc='lower right', fontsize=10, frameon=True, shadow=True)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()

# Save figure
plot_path = f"{RESULTS_ROOT}/bootstrap_distribution_vit_patch32_af.png"
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
print(f"Distribution plot saved: {os.path.basename(plot_path)}")
print(f"  Resolution: 300 DPI (publication quality)")
print(f"  Location: {plot_path}")

# Close figure to free memory
plt.close()

# =====================================================
# SECTION 6: SAVE RESULTS
# =====================================================

print("\n[STEP 6] Saving bootstrap results...")

# Prepare comprehensive results
final_results = {
    'model_information': {
        'model': metadata['model'],
        'methodology': metadata['methodology'],
        'phase': metadata['phase'],
        'experiment_id': metadata.get('experiment_id', 'run_02_patch32_FL'),
        'test_dataset': metadata['test_dataset'],
        'test_dataset_description': metadata['test_dataset_description']
    },
    'test_set_information': {
        'total_samples': len(y_true),
        'available_classes': available_classes,
        'missing_classes': missing_classes,
        'class_distribution': original_metrics.get('class_distribution', {})
    },
    'original_metrics': {
        'macro_f1': original_metrics['macro_f1'],
        'accuracy': original_metrics['accuracy'],
        'macro_precision': original_metrics['macro_precision'],
        'macro_recall': original_metrics['macro_recall']
    },
    'bootstrap_results': bootstrap_results,
    'interpretation': {
        'stability': stability,
        'ci_width': float(ci_width),
        'relative_ci_width_percent': float(relative_ci_width),
        'margin_of_error': float(margin_of_error),
        'bias': float(bias),
        'interpretation_text': interpretation
    },
    'paper_ready_text': {
        'inline_citation': f"macro F1 of {original_f1:.4f} (95% CI: [{ci_lower:.4f}, {ci_upper:.4f}])",
        'table_entry': f"{original_f1:.4f} [{ci_lower:.4f}, {ci_upper:.4f}]",
        'methods_text': f"Bootstrap confidence intervals (1000 iterations) were calculated to assess statistical reliability of performance metrics on the {len(y_true)}-sample test set."
    },
    'analysis_metadata': {
        'analysis_timestamp': datetime.now().strftime("%Y%m%d_%H%M%S"),
        'bootstrap_method': 'percentile',
        'confidence_level': 0.95,
        'n_iterations': 1000,
        'random_seed': 42
    }
}

# Save as JSON
results_path = f"{RESULTS_ROOT}/bootstrap_ci_results_vit_patch32_af.json"
with open(results_path, 'w') as f:
    json.dump(final_results, f, indent=2)

print(f"Bootstrap results saved: {os.path.basename(results_path)}")

# Save summary statistics as CSV for easy viewing
summary_df = pd.DataFrame({
    'Metric': ['Original F1', 'Bootstrap Mean', 'Bootstrap Median', 'Bootstrap Std',
               'CI Lower (95%)', 'CI Upper (95%)', 'CI Width', 'Margin of Error'],
    'Value': [original_f1, ci_mean, ci_median, ci_std,
              ci_lower, ci_upper, ci_width, margin_of_error]
})

csv_path = f"{RESULTS_ROOT}/bootstrap_summary_vit_patch32_af.csv"
summary_df.to_csv(csv_path, index=False, float_format='%.4f')
print(f"Summary statistics saved: {os.path.basename(csv_path)}")

file_size_json = os.path.getsize(results_path) / 1024
print(f"  JSON file size: {file_size_json:.1f} KB")

# =====================================================
# SECTION 7: PAPER-READY OUTPUT
# =====================================================

print("\n" + "=" * 70)
print("PAPER-READY RESULTS")
print("=" * 70)

print("\n1. INLINE CITATION (for Abstract/Results):")
print("-" * 70)
print(f"ViT-patch32 achieved a {final_results['paper_ready_text']['inline_citation']} on")
print(f"the apex-only test set (n={len(y_true)}), demonstrating peak single-frame")
print(f"performance with statistical validation.")

print("\n2. TABLE III UPDATE (Main Results):")
print("-" * 70)
print(f"Phase    Model        M1 F1 (95% CI)")
print(f"AF       ViT-p32      {final_results['paper_ready_text']['table_entry']}")

print("\n3. METHODS SECTION TEXT:")
print("-" * 70)
print(f"{final_results['paper_ready_text']['methods_text']}")

print("\n4. STATISTICAL DETAILS:")
print("-" * 70)
print(f"Bootstrap resampling (n=1000 iterations) yielded a mean F1 of {ci_mean:.4f}")
print(f"(SD={ci_std:.4f}) with 95% confidence interval [{ci_lower:.4f}, {ci_upper:.4f}].")
print(f"The confidence interval (width={ci_width:.4f}, {relative_ci_width:.1f}% of point")
print(f"estimate) indicates {stability.lower()} with the 28-sample apex-only test set.")

# =====================================================
# SECTION 8: SUMMARY
# =====================================================

print("\n" + "=" * 70)
print("BOOTSTRAP ANALYSIS COMPLETED - ViT-patch32 AF")
print("=" * 70)

print("\nKey findings:")
print(f"  Original macro F1: {original_f1:.4f}")
print(f"  95% Confidence Interval: [{ci_lower:.4f}, {ci_upper:.4f}]")
print(f"  Margin of error: ±{margin_of_error:.4f}")
print(f"  Stability: {stability}")

print("\nOutput files:")
print(f"  1. {os.path.basename(results_path)}")
print(f"  2. {os.path.basename(csv_path)}")
print(f"  3. {os.path.basename(plot_path)}")

print("\nRecommendation for camera-ready paper:")
print(f"  Update Table III with: {original_f1:.4f} ({ci_lower:.4f}-{ci_upper:.4f})")
print(f"  Both PoolFormer MFS and ViT AF now have statistical validation")
print(f"  Add comprehensive Methods text about bootstrap validation")

print("\n" + "=" * 70)

BOOTSTRAP CONFIDENCE INTERVAL ANALYSIS
Resampling-Based Statistical Validation - ViT-patch32 AF

[STEP 1] Loading predictions from Cell 1...
Predictions loaded successfully
  Model: ViT-patch32
  Phase: AF (Apex Frame)
  Test samples: 28
  Original Macro F1: 0.4235
  Available classes: 6
  Missing classes: ['fear']

[STEP 2] Implementing bootstrap resampling function...
Bootstrap function implemented
  Method: Percentile-based confidence intervals
  Resampling: With replacement, preserving sample size

[STEP 3] Running bootstrap analysis...
This may take 10-20 seconds for 1000 iterations
Bootstrap configuration:
  Iterations: 1000
  Confidence level: 95%
  Sample size: 28
  Random seed: 42
  Metric: Macro F1 (available classes only)


Bootstrap resampling: 100%|██████████| 1000/1000 [00:02<00:00, 449.29it/s]



Bootstrap analysis completed
  Bootstrap mean F1: 0.4220
  Bootstrap std: 0.1197
  95% CI: [0.2209, 0.6825]
  CI width: 0.4616

Comparison with original:
  Original F1: 0.4235
  Bootstrap mean: 0.4220
  Bias: -0.001480
  Assessment: Low bias, bootstrap distribution is centered

[STEP 4] Statistical interpretation...
Confidence interval analysis:
  Point estimate (original): 0.4235
  95% CI: [0.2209, 0.6825]
  Margin of error: ±0.2308
  Relative CI width: 109.0% of point estimate

Stability assessment: Low stability
  Wide confidence interval reflects test set size limitations

Performance reliability:
  Lower bound (0.2209) near or below 0.35 threshold
  Conclusion: Performance variability due to small test set

[STEP 5] Creating distribution visualization...
Distribution plot saved: bootstrap_distribution_vit_patch32_af.png
  Resolution: 300 DPI (publication quality)
  Location: /content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project/results/bootstrap_ci_results/